In [1]:
import cv2
import numpy as np
import os

# Use raw string for Windows paths
dataset_path = "/Users/arif/CVPR_FINAL_ASSIGNMENT2/dataset"

faces = []
labels = []
label_map = {}
current_label = 0


In [2]:
def augment(img):
    """Return a list of augmented images for LBPH training."""
    imgs = [img]
    
    # small rotations
    for angle in [-10, 10]:
        M = cv2.getRotationMatrix2D((100, 100), angle, 1.0)
        imgs.append(cv2.warpAffine(img, M, (200, 200)))
    
    # brightness adjustments
    imgs.append(cv2.convertScaleAbs(img, alpha=1.2, beta=20))  # brighter
    imgs.append(cv2.convertScaleAbs(img, alpha=0.8, beta=-20)) # darker
    
    return imgs


In [3]:
valid_ext = (".jpg", ".jpeg", ".png")

for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)

    if not os.path.isdir(person_path):
        continue

    label_map[current_label] = person_name

    for img_name in os.listdir(person_path):

        # skip non-image files
        if not img_name.lower().endswith(valid_ext):
            continue

        img_path = os.path.join(person_path, img_name)

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # skip unreadable images
        if img is None:
            print(f"Skipping: {img_path}")
            continue

        img = cv2.resize(img, (200, 200))
        img = cv2.equalizeHist(img)

        augmented_imgs = augment(img)
        faces.extend(augmented_imgs)
        labels.extend([current_label] * len(augmented_imgs))

    current_label += 1

faces = np.array(faces)
labels = np.array(labels)

print("Labels map:", label_map)
print("Total training images:", len(faces))


Labels map: {0: 'Tahmid', 1: 'Arif'}
Total training images: 500


In [4]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, labels)
print("Model trained successfully")


Model trained successfully


In [5]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)




while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces_detected = face_cascade.detectMultiScale(
        gray, scaleFactor=1.3, minNeighbors=5
    )

    for (x, y, w, h) in faces_detected:
        face_img = gray[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (200, 200))

        label, confidence = recognizer.predict(face_img)

        name = label_map[label] if confidence < 80 else "Unknown"

        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(
            frame,
            f"{name}",
            (x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (0,255,0),
            2
        )

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
